In [1]:
import os
import re
import pickle

import numpy as np
import pandas as pd

In [2]:
schic_path = '/n/netscratch/zhuang_lab/Lab/Peter/higashi_dnamerfish/schic_snm3cseq'

# Higashi on snm3C-seq from Liu/Ecker 2023

## Set up label_info.pickle (metadata) file

In [3]:
metadata_df = pd.read_csv('/n/home09/pren/higfiles/schic_metadata/CEMBA.m3C.Metadata.csv')
metadata_df


,cell,mCCCFrac,mCGFrac,mCHFrac,FinalmCReads,DissectionRegion,Plate,Col384,Row384,Slice,...,TransContact,TransRatio,Technology,InputReads,PassBasicQC,PlateNormCov,CEMBARegion,MajorRegion,SubRegion,CellGroup
0,CEMBA191126_9J_1-CEMBA191126_9J_2-A1-AD002,0.004221,0.704487,0.008457,3351057,HPF-2,CEMBA191126_9J_1,0,1,9,...,203408,0.117162,snm3C-seq,5325450,True,2.681377,9J,HPF,HIP,c5_c0_c0_c0_r3
1,CEMBA191126_9J_1-CEMBA191126_9J_2-A1-AD004,0.004694,0.711583,0.008305,7339138,HPF-2,CEMBA191126_9J_1,1,0,9,...,464628,0.121757,snm3C-seq,11527562,True,5.872473,9J,HPF,HIP,c5_c0_c1_c0_r3
2,CEMBA191126_9J_1-CEMBA191126_9J_2-A1-AD006,0.004626,0.712702,0.009139,1562166,HPF-2,CEMBA191126_9J_1,1,1,9,...,101302,0.125669,snm3C-seq,2600118,True,1.249980,9J,HPF,HIP,c5_c0_c0_c0_r3
3,CEMBA191126_9J_1-CEMBA191126_9J_2-A1-AD007,0.003694,0.662486,0.004810,1427822,HPF-2,CEMBA191126_9J_2,0,0,9,...,81962,0.111194,snm3C-seq,2278846,True,1.256558,9J,HPF,HIP,c11_c0_c1_c1_r8
4,CEMBA191126_9J_1-CEMBA191126_9J_2-A1-AD008,0.004966,0.713724,0.009948,1394985,HPF-2,CEMBA191126_9J_2,0,1,9,...,85557,0.119172,snm3C-seq,2372930,True,1.227659,9J,HPF,HIP,c5_c0_c0_c0_r3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175998,CEMBA3C_18A3C_R2_P3-4-B6-O8,0.005491,0.717344,0.010345,2444290,CB-2,CEMBA3C_18A3C_R2_P3,7,14,18,...,151285,0.119620,snm3C-seq,4331410,True,1.082704,18A3C,CB,CB,c3_c1_c1_c0_r1
175999,CEMBA3C_18A3C_R2_P3-4-B6-P19,0.005794,0.719815,0.010979,2176294,CB-2,CEMBA3C_18A3C_R2_P3,18,15,18,...,140641,0.125097,snm3C-seq,3909684,True,0.963994,18A3C,CB,CB,c3_c0_c0_c0_r1
176000,CEMBA3C_18A3C_R2_P3-4-B6-P20,0.004305,0.714725,0.008890,1334611,CB-2,CEMBA3C_18A3C_R2_P3,19,15,18,...,70487,0.103228,snm3C-seq,2243776,True,0.591169,18A3C,CB,CB,c3_c1_c1_c0_r1
176001,CEMBA3C_18A3C_R2_P3-4-B6-P7,0.004958,0.724756,0.011000,1794581,CB-2,CEMBA3C_18A3C_R2_P3,6,15,18,...,115189,0.125694,snm3C-seq,2998212,True,0.794914,18A3C,CB,CB,c1_c0_c0_c1_r4


In [6]:
mop_only_df = metadata_df.loc[metadata_df["SubRegion"] == "MOp"]
mop_only_df.cell.to_csv("/n/home09/pren/higfiles/schic_metadata/MoP_cellID.txt", sep = "\t")
mop_only_df


,cell,mCCCFrac,mCGFrac,mCHFrac,FinalmCReads,DissectionRegion,Plate,Col384,Row384,Slice,...,TransContact,TransRatio,Technology,InputReads,PassBasicQC,PlateNormCov,CEMBARegion,MajorRegion,SubRegion,CellGroup
10852,CEMBA3C_MOp2C_R1_P1-1-F3-A1,0.009651,0.742974,0.029639,1474669,Cortex-1,CEMBA3C_MOp2C_R1_P1,0,0,2,...,79290,0.107552,snm3C-seq,2195660,True,1.206128,2C,Isocortex,MOp,c2_c5_c2_c1_r0
10853,CEMBA3C_MOp2C_R1_P1-1-F3-A13,0.011412,0.770308,0.035168,1129118,Cortex-1,CEMBA3C_MOp2C_R1_P1,12,0,2,...,54667,0.097066,snm3C-seq,1681996,True,0.923503,2C,Isocortex,MOp,c8_c0_c2_c0_r0
10854,CEMBA3C_MOp2C_R1_P1-1-F3-A14,0.011827,0.780458,0.033843,917788,Cortex-1,CEMBA3C_MOp2C_R1_P1,13,0,2,...,47948,0.105289,snm3C-seq,1363894,True,0.750657,2C,Isocortex,MOp,c8_c0_c2_c1_r0
10855,CEMBA3C_MOp2C_R1_P1-1-F3-A2,0.010439,0.750732,0.027919,1152222,Cortex-1,CEMBA3C_MOp2C_R1_P1,1,0,2,...,59679,0.104111,snm3C-seq,1710864,True,0.942400,2C,Isocortex,MOp,c2_c5_c1_c0_r0
10856,CEMBA3C_MOp2C_R1_P1-1-F3-B1,0.011231,0.778385,0.031562,975792,Cortex-1,CEMBA3C_MOp2C_R1_P1,0,1,2,...,44811,0.092511,snm3C-seq,1450248,True,0.798098,2C,Isocortex,MOp,c2_c0_c2_c0_r0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169094,CEMBA3C_6A3C_R4_P2-3-M9-O6,0.008421,0.732946,0.026165,2129464,Cortex-4,CEMBA3C_6A3C_R4_P2,5,14,6,...,121234,0.114488,snm3C-seq,3662216,True,0.979625,6A3C,Isocortex,MOp,c2_c0_c0_c0_r1
169095,CEMBA3C_6A3C_R4_P2-3-M9-P17,0.006692,0.721449,0.006918,2428032,Cortex-4,CEMBA3C_6A3C_R4_P2,16,15,6,...,162912,0.139493,snm3C-seq,5942364,True,1.116976,6A3C,Isocortex,MOp,c4_c5_c0_c0_r2
169096,CEMBA3C_6A3C_R4_P2-3-M9-P18,0.008048,0.734579,0.023089,2052047,Cortex-4,CEMBA3C_6A3C_R4_P2,17,15,6,...,119196,0.116556,snm3C-seq,3453196,True,0.944010,6A3C,Isocortex,MOp,c2_c0_c2_c2_r1
169097,CEMBA3C_6A3C_R4_P2-3-M9-P5,0.009023,0.768067,0.028509,2209702,Cortex-4,CEMBA3C_6A3C_R4_P2,4,15,6,...,132743,0.121052,snm3C-seq,3810676,True,1.016537,6A3C,Isocortex,MOp,c8_c0_c0_c0_r1


In [5]:
cell_ids = [filename.split('.3C.')[0] for filename in os.listdir(os.path.join(schic_path, 'data_proc'))]
len(cell_ids)


17422

In [14]:
training_data_meta = pd.DataFrame(cell_ids, columns=['cell']).merge(metadata_df, on='cell', how='left')
training_data_meta = training_data_meta.rename(columns={'cell': 'Cell_ID'})
training_data_meta


,Cell_ID,mCCCFrac,mCGFrac,mCHFrac,FinalmCReads,DissectionRegion,Plate,Col384,Row384,Slice,...,TransContact,TransRatio,Technology,InputReads,PassBasicQC,PlateNormCov,CEMBARegion,MajorRegion,SubRegion,CellGroup
0,CEMBA3C_2A3C_R1_P2-1-I7-A1,0.009399,0.729415,0.027549,1626557,Cortex-1,CEMBA3C_2A3C_R1_P2,0,0,2,...,75218,0.089197,snm3C-seq,2827622,True,0.832431,2A3C,Isocortex,MOp,c2_c5_c2_c1_r0
1,CEMBA3C_2A3C_R1_P2-1-I7-A13,0.008281,0.776016,0.020019,2103603,Cortex-1,CEMBA3C_2A3C_R1_P2,12,0,2,...,119997,0.109766,snm3C-seq,3676988,True,1.076571,2A3C,Isocortex,MOp,c18_c6_c2_c2_r0
2,CEMBA3C_2A3C_R1_P2-1-I7-A14,0.008577,0.797123,0.031574,772413,Cortex-1,CEMBA3C_2A3C_R1_P2,13,0,2,...,39263,0.098443,snm3C-seq,1341568,True,0.395301,2A3C,Isocortex,MOp,c9_c1_c0_c1_r0
3,CEMBA3C_2A3C_R1_P2-1-I7-A2,0.009188,0.770996,0.027978,2000611,Cortex-1,CEMBA3C_2A3C_R1_P2,1,0,2,...,94157,0.090362,snm3C-seq,3519560,True,1.023862,2A3C,Isocortex,MOp,c6_c1_c0_c0_r0
4,CEMBA3C_2A3C_R1_P2-1-I7-B1,0.009337,0.763041,0.027664,1929775,Cortex-1,CEMBA3C_2A3C_R1_P2,0,1,2,...,84027,0.083506,snm3C-seq,3424452,True,0.987610,2A3C,Isocortex,MOp,c6_c1_c1_c1_r0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17417,CEMBA3C_MOp3C_R1_P1-5-A17-L10,0.006336,0.678327,0.008811,1951897,Cortex-2,CEMBA3C_MOp3C_R1_P1,9,11,3,...,121653,0.120909,snm3C-seq,3556098,True,0.816423,3C,Isocortex,MOp,c11_c0_c2_c0_r6
17418,CEMBA3C_MOp3C_R1_P1-5-A17-L21,0.005995,0.706599,0.008036,2910927,Cortex-2,CEMBA3C_MOp3C_R1_P1,20,11,3,...,207907,0.137427,snm3C-seq,5574482,True,1.217558,3C,Isocortex,MOp,c15_c2_c0_c0_r3
17419,CEMBA3C_MOp3C_R1_P1-5-A17-L22,0.008308,0.735115,0.023973,2866449,Cortex-2,CEMBA3C_MOp3C_R1_P1,21,11,3,...,193141,0.130179,snm3C-seq,5398650,True,1.198954,3C,Isocortex,MOp,c2_c1_c1_c1_r0
17420,CEMBA3C_MOp3C_R1_P1-5-A17-L9,0.009818,0.743488,0.029964,2164210,Cortex-2,CEMBA3C_MOp3C_R1_P1,8,11,3,...,146015,0.130695,snm3C-seq,4245592,True,0.905228,3C,Isocortex,MOp,c2_c1_c2_c0_r0


In [7]:
training_data_meta.columns

Index(['cell', 'mCCCFrac', 'mCGFrac', 'mCHFrac', 'FinalmCReads',
       'DissectionRegion', 'Plate', 'Col384', 'Row384', 'Slice', 'Sample',
       'CisLongContact', 'CisLongRatio', 'TransContact', 'TransRatio',
       'Technology', 'InputReads', 'PassBasicQC', 'PlateNormCov',
       'CEMBARegion', 'MajorRegion', 'SubRegion', 'CellGroup'],
      dtype='object')

In [24]:
label_info = {k:np.asarray(training_data_meta[k]) for k in training_data_meta.columns}
label_info_path = '/n/home09/pren/higfiles/schic_metadata/label_info.pickle'
with open(label_info_path, 'wb') as handle:
    pickle.dump(label_info, handle)


## Set up config file

In [29]:
config = "/n/home09/pren/higfiles/configs/config_liu_ecker.JSON"
config_info = {
    "data_dir": schic_path, # where the data is, the data has to be named as data.txt (v1) or filelist.txt (v2) or the code can't find it
    "label_path": label_info_path,
    "structured": True,
    "input_format": 'higashi_v2',
    "temp_dir": "/n/netscratch/zhuang_lab/Lab/Peter/higashi_dnamerfish/embeddings_liu_ecker_temp/", # where to store model temp files
    "genome_reference_path": "/n/home09/pren/higfiles/params/chromInfo.txt",
    "cytoband_path": "/n/home09/pren/higfiles/params/cytoBand.txt",
    "chrom_list": ['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
       'chr17', 'chr18', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr9',
       'chr16', 'chr19', 'chr2', 'chr8', "chrX"],
    "resolution": 500000, # 500 kbp?
    "resolution_cell": 500000,
    "resolution_fh": [500000],
    "embedding_name": "test_snm3cseq",
    "minimum_distance": 500000,
    "maximum_distance": -1,
    "local_transfer_range": 0,
    "loss_mode": "zinb",
    "dimensions": 100, # can be adjusted later
    "impute_list":['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
       'chr17', 'chr18', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr9',
       'chr16', 'chr19', 'chr2', 'chr8', "chrX"],
    "neighbor_num": 5,
    "cpu_num": 10,
    "gpu_num": 1,
    "embedding_epoch":50, # this can be adjusted
    "correct_be_impute": True,
    "header_included": True,
}

# save the config file to the current directory
import json
with open(config, "w") as f:
    json.dump(config_info, f, indent = 6)

## Run Higashi

In [2]:
import sys
sys.path.append('/n/home09/pren/Higashi')

In [3]:
from higashi.Higashi_wrapper import *


In [ ]:
# Initialize the Higashi instance
higashi_model = Higashi(config)

# Data processing (only needs to be run for once)
higashi_model.process_data()
higashi_model.prep_model()


## Stage 1 training

In [ ]:
# Stage 1 training
higashi_model.train_for_embeddings()


In [ ]:
# Visualize initial embedding results
cell_embeddings = higashi_model.fetch_cell_embeddings()
print (cell_embeddings.shape)

from umap import UMAP
import seaborn as sns
import matplotlib.pyplot as plt

vec = UMAP(n_components=2, n_neighbors=25, random_state=0).fit_transform(cell_embeddings)
cell_type = higashi_model.label_info['CellGroup']
batch = higashi_model.label_info['Sample']
fig = plt.figure(figsize=(14, 5))
ax = plt.subplot(1, 2, 1)
sns.scatterplot(x=vec[:, 0], y=vec[:, 1], hue=cell_type, ax=ax, s=5, alpha=0.8, linewidth=0, hue_order=['GM12878','IMR90','HFF', 'HAP1', 'H1Esc'])
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles=handles, labels=labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., ncol=1)
ax = plt.subplot(1, 2, 2)
sns.scatterplot(x=vec[:, 0], y=vec[:, 1], hue=batch, ax=ax, s=5, alpha=0.8, linewidth=0)
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles=handles, labels=labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., ncol=1)
plt.tight_layout()
plt.show()


## Stage 2 training and imputation without neighbors

In [ ]:
higashi_model.train_for_imputation_nbr_0()
higashi_model.impute_no_nbr()

## Stage 3 training and imputation with neighbors

In [ ]:
higashi_model.train_for_imputation_with_nbr()
higashi_model.impute_with_nbr()

In [ ]:
# Visualize final embedding results
cell_embeddings = higashi_model.fetch_cell_embeddings()
print (cell_embeddings.shape)

from sklearn.decomposition import PCA

cell_type = higashi_model.label_info['CellGroup']
fig = plt.figure(figsize=(14, 5))
ax = plt.subplot(1, 2, 1)
vec = PCA(n_components=2).fit_transform(cell_embeddings)
sns.scatterplot(x=vec[:, 0], y=vec[:, 1], hue=cell_type, ax=ax, s=6, linewidth=0)
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles=handles, labels=labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., ncol=1)
ax = plt.subplot(1, 2, 2)
vec = UMAP(n_components=2).fit_transform(cell_embeddings)
sns.scatterplot(x=vec[:, 0], y=vec[:, 1], hue=cell_type, ax=ax, s=6, linewidth=0)
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles=handles, labels=labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., ncol=1)
plt.tight_layout()
plt.show()


## Visualize imputation results

In [ ]:
count = 0
fig = plt.figure(figsize=(6, 2*5))
for id_ in np.random.randint(0, 620, 5):
    ori, nbr0, nbr5 = higashi_model.fetch_map("chr3", id_)
    count += 1
    ax = plt.subplot(5, 3, count * 3 - 2)
    ax.imshow(ori.toarray(), cmap='Reds', vmin=0.0, vmax=np.quantile(ori.data, 0.6))
    ax.set_xticks([], [])
    ax.set_yticks([], [])
    if count == 1:
        ax.set_title("raw")
    
    ax = plt.subplot(5, 3, count * 3 - 1)
    ax.imshow(nbr0.toarray(), cmap='Reds', vmin=0.0, vmax=np.quantile(nbr0.data, 0.95))
    ax.set_xticks([], [])
    ax.set_yticks([], [])
    if count == 1:
        ax.set_title("higashi, k=0")
    
    ax = plt.subplot(5, 3, count * 3)
    ax.imshow(nbr5.toarray(), cmap='Reds', vmin=0.0, vmax=np.quantile(nbr5.data, 0.95))
    ax.set_xticks([], [])
    ax.set_yticks([], [])
    if count == 1:
        ax.set_title("higashi, k=5")
    
plt.tight_layout()

# Higashi on DNA MERFISH from Liu/Zhuang 2023

In [8]:
fish_path = '/n/netscratch/zhuang_lab/Lab/Peter/higashi_dnamerfish/multiplexed_fish'

In [9]:
cell_df = pd.read_csv(os.path.join(fish_path, '4DNESMTNNB3N/4DNFIA7FUW8Y.csv'), skiprows=22)
cell_df = cell_df.rename(columns={'##columns=(Cell_ID': 'Cell_ID', 'Zfp804b)': 'Zfp804b'})
cell_df


,Cell_ID,FOV_ID,cell_volume_from_merlin,cell_center_x_global,cell_center_y_global,RNA_experiment_ID,Sample_ID,cluster_subclass,cluster_class,neuron_identity,...,Unc13c,Unc5b,Unc5d,Ust,Vipr2,Vtn,Vwc2,Wipf3,Wnt7b,Zfp804b
0,103489270985795086009915676188840872439,0,403.121799,-5984.643199,3962.926100,20220304,C57BL/6_M_1_MOp_1,Astro,Astro,Non-Neuronal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,113811790893231448949429397438012891173,0,1345.443053,-6046.845801,3943.378100,20220304,C57BL/6_M_1_MOp_1,other,other,other,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,119547840309256440899855144626134392385,0,866.273544,-6009.207800,3915.406099,20220304,C57BL/6_M_1_MOp_1,other,other,other,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,120849314423504941338008541020153695260,0,1186.181156,-6114.885802,3938.572100,20220304,C57BL/6_M_1_MOp_1,other,other,other,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,125550627244524118102850004242680097855,0,281.811702,-6190.485804,3985.876101,20220304,C57BL/6_M_1_MOp_1,Peri,Peri,Non-Neuronal,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17851,89740340849279220555487673174291177981,167,746.108561,3028.620050,-2372.662093,20220304,C57BL/6_M_1_MOp_1,L5/6 NP,Gluta,Neuronal,...,5.0,0.0,0.0,1.0,1.0,0.0,0.0,8.0,0.0,1.0
17852,90550019712359509388050476283813007625,167,931.565237,3093.798051,-2484.658096,20220304,C57BL/6_M_1_MOp_1,L6 CT,Gluta,Neuronal,...,1.0,0.0,0.0,2.0,0.0,0.0,0.0,30.0,0.0,0.0
17853,91790428117198932838975210302149417238,167,146.709916,3073.926051,-2429.902094,20220304,C57BL/6_M_1_MOp_1,Astro,Astro,Non-Neuronal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17854,95847719459140129344505526245510134602,167,264.662706,3019.332049,-2441.458095,20220304,C57BL/6_M_1_MOp_1,L6 CT,Gluta,Neuronal,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,1.0


In [10]:
cell_ids_merfish = [re.search('cell\d+', filename).group()[4:] for filename in os.listdir(os.path.join(fish_path, 'merfish_contacts'))]
len(cell_ids_merfish)


2413

In [11]:
training_data_merfish_meta = pd.DataFrame(cell_ids_merfish, columns=['Cell_ID']).merge(cell_df, on='Cell_ID', how='left')
training_data_merfish_meta

,Cell_ID,FOV_ID,cell_volume_from_merlin,cell_center_x_global,cell_center_y_global,RNA_experiment_ID,Sample_ID,cluster_subclass,cluster_class,neuron_identity,...,Unc13c,Unc5b,Unc5d,Ust,Vipr2,Vtn,Vwc2,Wipf3,Wnt7b,Zfp804b
0,100143354915909514340563966591344282617,84.0,852.211658,2475.098099,-1024.199995,20220304.0,C57BL/6_M_1_MOp_1,L2/3 IT,Gluta,Neuronal,...,2.0,2.0,0.0,1.0,0.0,0.0,0.0,33.0,0.0,1.0
1,100444929727335622562402977796883679092,8.0,844.089198,-5802.457800,4135.278100,20220304.0,C57BL/6_M_1_MOp_1,Endo,Endo,Non-Neuronal,...,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100636001562801092372127493520169017848,75.0,598.349001,-6372.189898,5324.277907,20220304.0,C57BL/6_M_1_MOp_1,VLMC,VLMC,Non-Neuronal,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100641667178165970074422975257012665391,84.0,762.882093,2494.970100,-1057.625996,20220304.0,C57BL/6_M_1_MOp_1,Endo,Endo,Non-Neuronal,...,2.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,100645494773040598441840741335181821324,81.0,917.879399,2390.604102,-437.051995,20220304.0,C57BL/6_M_1_MOp_1,Astro,Astro,Non-Neuronal,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408,99732441040257909568179107546777692281,85.0,1189.559867,2588.552102,-950.289998,20220304.0,C57BL/6_M_1_MOp_1,Sst,GABA,Neuronal,...,13.0,2.0,6.0,7.0,1.0,1.0,13.0,17.0,1.0,0.0
2409,99760521419024717766427788907150197811,84.0,554.913079,2633.588103,-1030.841995,20220304.0,C57BL/6_M_1_MOp_1,L5 IT,Gluta,Neuronal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2410,99806661526131685558261006620437651282,72.0,625.749255,-6655.781900,5085.343906,20220304.0,C57BL/6_M_1_MOp_1,Endo,Endo,Non-Neuronal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2411,99895158179073067511837782015760741030,9.0,498.213556,-5864.665801,3896.452099,20220304.0,C57BL/6_M_1_MOp_1,Micro,Micro,Non-Neuronal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
training_data_meta_merged = pd.merge(training_data_meta, training_data_merfish_meta, how='outer', on='Cell_ID')
training_data_meta_merged


,Cell_ID,mCCCFrac,mCGFrac,mCHFrac,FinalmCReads,DissectionRegion,Plate,Col384,Row384,Slice,...,Unc13c,Unc5b,Unc5d,Ust,Vipr2,Vtn,Vwc2,Wipf3,Wnt7b,Zfp804b
0,CEMBA3C_2A3C_R1_P2-1-I7-A1,0.009399,0.729415,0.027549,1626557.0,Cortex-1,CEMBA3C_2A3C_R1_P2,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CEMBA3C_2A3C_R1_P2-1-I7-A13,0.008281,0.776016,0.020019,2103603.0,Cortex-1,CEMBA3C_2A3C_R1_P2,12.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CEMBA3C_2A3C_R1_P2-1-I7-A14,0.008577,0.797123,0.031574,772413.0,Cortex-1,CEMBA3C_2A3C_R1_P2,13.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CEMBA3C_2A3C_R1_P2-1-I7-A2,0.009188,0.770996,0.027978,2000611.0,Cortex-1,CEMBA3C_2A3C_R1_P2,1.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CEMBA3C_2A3C_R1_P2-1-I7-B1,0.009337,0.763041,0.027664,1929775.0,Cortex-1,CEMBA3C_2A3C_R1_P2,0.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19830,99732441040257909568179107546777692281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.0,2.0,6.0,7.0,1.0,1.0,13.0,17.0,1.0,0.0
19831,99760521419024717766427788907150197811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
19832,99806661526131685558261006620437651282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
19833,99895158179073067511837782015760741030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
print([col for col in training_data_meta_merged.columns])

['Cell_ID', 'mCCCFrac', 'mCGFrac', 'mCHFrac', 'FinalmCReads', 'DissectionRegion', 'Plate', 'Col384', 'Row384', 'Slice', 'Sample', 'CisLongContact', 'CisLongRatio', 'TransContact', 'TransRatio', 'Technology', 'InputReads', 'PassBasicQC', 'PlateNormCov', 'CEMBARegion', 'MajorRegion', 'SubRegion', 'CellGroup', 'FOV_ID', 'cell_volume_from_merlin', 'cell_center_x_global', 'cell_center_y_global', 'RNA_experiment_ID', 'Sample_ID', 'cluster_subclass', 'cluster_class', 'neuron_identity', '1700022I11Rik', '1810046K07Rik', '5031425F14Rik', '5730522E02Rik', 'Acta2', 'Adam2', 'Adamts2', 'Adamts4', 'Adra1b', 'Alk', 'Ankfn1', 'Ano4', 'Aqp4', 'Asic4', 'B4galnt2', 'B4galnt3', 'Barx2', 'Bcl11b', 'Bdnf', 'Bgn', 'Blnk', 'Bmpr1b', 'Brinp3', 'C1ql3', 'C1qtnf7', 'Cacng5', 'Calb2', 'Camk2d', 'Car3', 'Cbln2', 'Cbln4', 'Ccbe1', 'Ccdc162', 'Ccdc3', 'Ccdc80', 'Ccnb1', 'Cd14', 'Cd24a', 'Cdca7', 'Cdcp1', 'Cdh12', 'Cdh13', 'Cdh20', 'Cdh9', 'Ceacam9', 'Cemip', 'Chat', 'Chn2', 'Chodl', 'Chrm2', 'Chrna2', 'Cldn5', 'Clr

In [20]:
training_data_meta_merged['neuron_identity']

0                 NaN
1                 NaN
2                 NaN
3                 NaN
4                 NaN
             ...     
19830        Neuronal
19831        Neuronal
19832    Non-Neuronal
19833    Non-Neuronal
19834        Neuronal
Name: neuron_identity, Length: 19835, dtype: object

In [21]:
training_data_meta_merged['cluster_class']

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
         ...  
19830     GABA
19831    Gluta
19832     Endo
19833    Micro
19834    Gluta
Name: cluster_class, Length: 19835, dtype: object

In [22]:
def id_celltype(celltype):
    if celltype == 'GABA':
        return 'Gaba'
    elif celltype == 'Gluta':
        return 'Glut'
    else:
        return 'NN'

In [25]:
training_data_meta_merged['NeuroTransmitters'] = training_data_meta_merged['cluster_class'].apply(id_celltype)
training_data_meta_merged


,Cell_ID,mCCCFrac,mCGFrac,mCHFrac,FinalmCReads,DissectionRegion,Plate,Col384,Row384,Slice,...,Unc5b,Unc5d,Ust,Vipr2,Vtn,Vwc2,Wipf3,Wnt7b,Zfp804b,NeuroTransmitters
0,CEMBA3C_2A3C_R1_P2-1-I7-A1,0.009399,0.729415,0.027549,1626557.0,Cortex-1,CEMBA3C_2A3C_R1_P2,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NN
1,CEMBA3C_2A3C_R1_P2-1-I7-A13,0.008281,0.776016,0.020019,2103603.0,Cortex-1,CEMBA3C_2A3C_R1_P2,12.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NN
2,CEMBA3C_2A3C_R1_P2-1-I7-A14,0.008577,0.797123,0.031574,772413.0,Cortex-1,CEMBA3C_2A3C_R1_P2,13.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NN
3,CEMBA3C_2A3C_R1_P2-1-I7-A2,0.009188,0.770996,0.027978,2000611.0,Cortex-1,CEMBA3C_2A3C_R1_P2,1.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NN
4,CEMBA3C_2A3C_R1_P2-1-I7-B1,0.009337,0.763041,0.027664,1929775.0,Cortex-1,CEMBA3C_2A3C_R1_P2,0.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19830,99732441040257909568179107546777692281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,6.0,7.0,1.0,1.0,13.0,17.0,1.0,0.0,Gaba
19831,99760521419024717766427788907150197811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Glut
19832,99806661526131685558261006620437651282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NN
19833,99895158179073067511837782015760741030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NN
